In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [3]:
# Step 1: Load the dataset
data = pd.read_csv('https://raw.githubusercontent.com/Leonartinus/0-9_ASL_recognition/main/Leo0-9.csv')

# Step 2: Prepare the data
# Split the dataset into features (X) and labels (y)
X = data.drop('label', axis=1).values  # Features
y = data['label'].values  # Labels

# # Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
# Step 3: Build the neural network
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(X.shape[1], 128)
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, 32)
        self.output_layer = nn.Linear(32, 10)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.dropout(x)
        x = self.relu(self.layer2(x))
        x = self.dropout(x)
        x = self.relu(self.layer3(x))
        x = self.output_layer(x)
        return self.softmax(x)

In [5]:

model = NeuralNetwork()

# Step 4: Compile the model
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 5: Train the model
num_epochs = 100
batch_size = 32

# DataLoader for batching
train_data = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/100, Loss: 1.9277468919754028
Epoch 2/100, Loss: 1.643654704093933
Epoch 3/100, Loss: 1.6667170524597168
Epoch 4/100, Loss: 1.6549733877182007
Epoch 5/100, Loss: 1.6310149431228638
Epoch 6/100, Loss: 1.532344937324524
Epoch 7/100, Loss: 1.5450729131698608
Epoch 8/100, Loss: 1.5513701438903809
Epoch 9/100, Loss: 1.5451979637145996
Epoch 10/100, Loss: 1.5794113874435425
Epoch 11/100, Loss: 1.5068567991256714
Epoch 12/100, Loss: 1.5461632013320923
Epoch 13/100, Loss: 1.4714703559875488
Epoch 14/100, Loss: 1.5046061277389526
Epoch 15/100, Loss: 1.4615381956100464
Epoch 16/100, Loss: 1.5018301010131836
Epoch 17/100, Loss: 1.461224913597107
Epoch 18/100, Loss: 1.5459623336791992
Epoch 19/100, Loss: 1.461295247077942
Epoch 20/100, Loss: 1.6415961980819702
Epoch 21/100, Loss: 1.5029597282409668
Epoch 22/100, Loss: 1.5445410013198853
Epoch 23/100, Loss: 1.5734834671020508
Epoch 24/100, Loss: 1.4634041786193848
Epoch 25/100, Loss: 1.544851303100586
Epoch 26/100, Loss: 1.5628414154052734


In [6]:
# Step 6: Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    accuracy = accuracy_score(y_test, predicted)
    print(f'Test accuracy: {accuracy}')

Test accuracy: 0.9843641417651147


In [8]:
torch.save(model, 'leo_model.pt')

from google.colab import files

# Download the file
files.download('leo_model.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>